In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response = llm.invoke("The first person to land on earth was ...")
print(response.content)

I think there might be some confusion.  The first person to land on Earth would be the first human to be born on the planet. However, I believe you might be referring to the first person to walk on the Moon or another celestial body.

If that's the case, the first person to walk on the Moon was Neil Armstrong, an American astronaut who landed on July 20, 1969, as part of the Apollo 11 mission.

If you could provide more context or clarify your question, I'd be happy to try and assist you further.
